In [2]:
!pip install peft bitsandbytes accelerate huggingface_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 30.4 MB/s eta 0:00:00:00:0100:01


In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
# Add these tokens in Kaggle's "Secrets" tab (Add-ons -> Secrets)
login(token=user_secrets.get_secret("HF_TOKEN"))

In [4]:
import torch
import json
import re
import os
import random
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from peft import PeftModel
from datetime import datetime

# ==========================================
# 🎯 SELECT THE MODEL TO EVALUATE
# ==========================================
ACTIVE_MODEL = "phi4_qlora"
# ACTIVE_MODEL = "phi4_lora"
# ACTIVE_MODEL = "qwen3_qlora"
# ACTIVE_MODEL = "qwen3_lora"
# ACTIVE_MODEL = "gemma3_qlora"
# ACTIVE_MODEL = "gemma3_lora"

# ==========================================
# ⚙️  PATHS — edit dataset name to match yours
# ==========================================
ADAPTER_DATASET = "/kaggle/input/datasets/shreyashgaurgla/nyaya-adapters"   # ← change to your adapter dataset path
DATA_DATASET    = "/kaggle/input/datasets/shreyashgaurgla/nyaya-llm-dataset"  # ← change to your data dataset path

CONFIGS = {
    "phi4_qlora":   {"base_model": "microsoft/Phi-4-mini-instruct", "adapter_dir": f"{ADAPTER_DATASET}/qlora_phase1_phi4_mini/qlora_phase1_phi4_mini"},
    "phi4_lora":    {"base_model": "microsoft/Phi-4-mini-instruct", "adapter_dir": f"{ADAPTER_DATASET}/lora_phase1_phi4_mini/lora_phase1_phi4_mini"},
    "qwen3_qlora":  {"base_model": "Qwen/Qwen3-4B-Instruct-2507",  "adapter_dir": f"{ADAPTER_DATASET}/qlora_phase1_qwen3_4b/qlora_phase1_qwen3_4b"},
    "qwen3_lora":   {"base_model": "Qwen/Qwen3-4B-Instruct-2507",  "adapter_dir": f"{ADAPTER_DATASET}/lora_phase1_qwen3_4b/lora_phase1_qwen3_4b"},
    "gemma3_qlora": {"base_model": "google/gemma-3-4b-it",          "adapter_dir": f"{ADAPTER_DATASET}/qlora_phase1_gemma3_4b/qlora_phase1_gemma3_4b"},
    "gemma3_lora":  {"base_model": "google/gemma-3-4b-it",          "adapter_dir": f"{ADAPTER_DATASET}/lora_phase1_gemma3_4b/lora_phase1_gemma3_4b"},
}

TEST_DATA_PATH = f"{DATA_DATASET}/all_acts_test.jsonl"
NUM_SAMPLES    = 150
SEED           = 42
RESULTS_DIR    = "/kaggle/working/results"
JUDGE_RETRIES  = 3

# ==========================================
# 🏷️  TASK TYPE DETECTOR
# ==========================================
def detect_task_type(instruction: str) -> str:
    il = instruction.lower()
    if il.startswith("explain"):
        return "Explanation"
    elif il.startswith("summarize"):
        return "Summarization"
    elif il.startswith("what does"):
        return "Direct Q&A"
    elif il.startswith("under which act"):
        return "Act Identification"
    else:
        return "Other"

# ==========================================
# 📦  LOAD TEST SAMPLES
# ==========================================
def load_test_samples(filepath: str, num_samples: int) -> list:
    with open(filepath, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    random.seed(SEED)
    return random.sample(data, min(num_samples, len(data)))

# ==========================================
# 🤖  GENERATION
# ==========================================
def generate_response(model, tokenizer, instruction: str) -> str:
    prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_output.split("### Response:\n")[-1].strip()

# ==========================================
# 🧑‍⚖️  JUDGE — HuggingFace (no Ollama needed)
# ==========================================
JUDGE_PROMPT_TEMPLATE = """You are a strict legal AI evaluator. Score the MODEL ANSWER against the REFERENCE ANSWER.

SCORING RUBRIC:
  5 - Perfect. Legally accurate, complete, no errors.
  4 - Good. Correct core content, minor omissions or slight imprecision.
  3 - Acceptable. Gets the general idea but misses important legal details.
  2 - Poor. Partially correct but contains notable legal errors.
  1 - Wrong. Fabricated law, completely incorrect, or irrelevant answer.

IMPORTANT:
- Score based on semantic correctness, NOT exact wording match.
- The reference may be long. The model only needs to capture the key legal meaning.
- If the model answer is legally equivalent to the reference but phrased differently, score it high.

QUESTION:
{instruction}

REFERENCE ANSWER:
{reference}

MODEL ANSWER:
{prediction}

Respond ONLY with a valid JSON object, nothing else:
{{"score": <int 1-5>, "reasoning": "<one concise sentence>"}}"""

judge_pipe = None

def load_judge():
    global judge_pipe
    print("Loading judge model (Qwen2.5-7B 4-bit)...")

    judge_bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4"
    )

    judge_model = AutoModelForCausalLM.from_pretrained(
        "Qwen/Qwen2.5-7B-Instruct",
        quantization_config=judge_bnb,
        device_map="auto",
        torch_dtype=torch.float16
    )
    judge_model.generation_config.max_length = None

    judge_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

    judge_pipe = pipeline(
        "text-generation",
        model=judge_model,
        tokenizer=judge_tokenizer,
    )
        # clears the conflicting defaults
    judge_pipe.model.generation_config.max_length = None
    judge_pipe.model.generation_config.min_length = 0
    print("Judge loaded.\n")

def judge_score(instruction: str, reference: str, prediction: str) -> tuple:
    prompt = JUDGE_PROMPT_TEMPLATE.format(
        instruction=instruction,
        reference=reference[:600],
        prediction=prediction[:600]
    )

    for attempt in range(JUDGE_RETRIES):
        try:
            output = judge_pipe(
                prompt,
                max_new_tokens=150,
                min_new_tokens=10,
                do_sample=False,
                return_full_text=False,
                pad_token_id=judge_pipe.tokenizer.eos_token_id
            )
            response = output[0]["generated_text"].strip()

            # Strip markdown fences
            response = re.sub(r"```(?:json)?", "", response).strip()

            if not response:
                raise ValueError("Empty response from judge")

            match = re.search(r"\{.*?\}", response, re.DOTALL)
            if not match:
                raise ValueError(f"No JSON found. Raw: {response[:150]}")

            parsed = json.loads(match.group())
            score  = int(parsed["score"])

            if not (1 <= score <= 5):
                raise ValueError(f"Score out of range: {score}")

            return score, parsed.get("reasoning", "")

        except Exception as e:
            print(f"      ⚠️  Judge attempt {attempt + 1} failed: {e}")
            if attempt == JUDGE_RETRIES - 1:
                return 0, "Judge error — skipped"

    return 0, "Judge error — skipped"

# ==========================================
# 📊  SUMMARY PRINTER
# ==========================================
def print_summary(results: list, model_name: str):
    valid = [r for r in results if r["score"] > 0]

    if not valid:
        print("No valid scores to summarise.")
        return

    overall_avg = sum(r["score"] for r in valid) / len(valid)

    by_task    = defaultdict(list)
    score_dist = defaultdict(int)

    for r in valid:
        by_task[r["task_type"]].append(r["score"])
        score_dist[r["score"]] += 1

    print("\n" + "=" * 60)
    print(f"📊  PHASE 1 SUMMARY — {model_name.upper()}")
    print("=" * 60)
    print(f"  Scored      : {len(valid)} / {len(results)} samples")
    print(f"  Overall avg : {overall_avg:.2f} / 5.0")

    print("\n  Score distribution:")
    for s in range(5, 0, -1):
        count = score_dist.get(s, 0)
        bar   = "█" * count
        print(f"    {s}/5  {bar} ({count})")

    print("\n  By task type:")
    for task in sorted(by_task.keys()):
        scores = by_task[task]
        avg    = sum(scores) / len(scores)
        print(f"    {task:<20} avg={avg:.2f}  (n={len(scores)})")

    print("=" * 60)

# ==========================================
# 🚀  MAIN
# ==========================================
def main():
    cfg = CONFIGS[ACTIVE_MODEL]
    os.makedirs(RESULTS_DIR, exist_ok=True)
    output_file = f"{RESULTS_DIR}/phase1_eval_{ACTIVE_MODEL}.json"

    print(f"\n🚀 Evaluating: {ACTIVE_MODEL.upper()}")
    print("Loading 4-bit quantization config...")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    print(f"Loading base model: {cfg['base_model']}...")
    base_model = AutoModelForCausalLM.from_pretrained(
        cfg["base_model"],
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=("qwen" in ACTIVE_MODEL),
        torch_dtype=torch.float16
    )

    print(f"Loading LoRA adapter: {cfg['adapter_dir']}...")
    try:
        model = PeftModel.from_pretrained(base_model, cfg["adapter_dir"])
        model.eval()
    except Exception as e:
        print(f"\n❌ Could not load adapter. Did training finish?\n{e}")
        return

    tokenizer = AutoTokenizer.from_pretrained(
        cfg["base_model"],
        trust_remote_code=("qwen" in ACTIVE_MODEL)
    )

    # Load judge after main model to avoid VRAM conflict
    load_judge()

    print(f"\nLoading {NUM_SAMPLES} test samples (seed={SEED})...")
    samples = load_test_samples(TEST_DATA_PATH, NUM_SAMPLES)

    print(f"\n{'='*60}")
    print(f"⚖️   PHASE 1 EVALUATION: {ACTIVE_MODEL.upper()}")
    print(f"{'='*60}\n")

    results = []

    for i, sample in enumerate(samples, 1):
        instruction  = sample["instruction"]
        ground_truth = sample["output"]
        task_type    = detect_task_type(instruction)

        print(f"[{i:02d}/{NUM_SAMPLES}] [{task_type}]")
        print(f"  Prompt : {instruction[:80]}...")

        prediction = generate_response(model, tokenizer, instruction)
        print(f"  Answer : {prediction[:80]}...")

        score, reasoning = judge_score(instruction, ground_truth, prediction)
        print(f"  Score  : {score}/5  — {reasoning}\n")

        results.append({
            "model":        ACTIVE_MODEL,
            "task_type":    task_type,
            "instruction":  instruction,
            "ground_truth": ground_truth,
            "prediction":   prediction,
            "score":        score,
            "reasoning":    reasoning,
            "timestamp":    datetime.now().isoformat()
        })

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"💾 Raw results saved → {output_file}")
    print_summary(results, ACTIVE_MODEL)


if __name__ == "__main__":
    main()


🚀 Evaluating: PHI4_QLORA
Loading 4-bit quantization config...
Loading base model: microsoft/Phi-4-mini-instruct...


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
This model config has set a `rope_parameters['original_max_position_embeddings']` field, to be used together with `max_position_embeddings` to determine a scaling factor. Please set the `factor` field of `rope_parameters`with this ratio instead -- we recommend the use of this field over `original_max_position_embeddings`, as it is compatible with most model architectures.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

  Answer : Indian Evidence Act, 1872
Chapter 8: 
Section 117: Proof of fact by oral evidenc...
  Score  : 2/5  — The model answer incorrectly interprets Section 117 as about oral evidence instead of estoppel.

[149/150] [Explanation]
  Prompt : Explain Section 6 of the Indian Divorce Act, 1869....


Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `min_new_tokens` (=10) and `min_length`(=0) seem to have been set. `min_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Answer : Indian Divorce Act, 1869
Chapter 2: 
Section 6: Foreign judgments

In any suit u...
  Score  : 2/5  — The model answer incorrectly interprets Section 6 and introduces irrelevant information about foreign judgments and alimony.

[150/150] [Summarization]
  Prompt : Summarize Section 105 of Indian Penal Code, 1860....


Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `min_new_tokens` (=10) and `min_length`(=0) seem to have been set. `min_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Answer : The imprisonment which a Court of Session may direct in default of payment of a ...
  Score  : 1/5  — The model answer is completely unrelated and incorrect.

💾 Raw results saved → /kaggle/working/results/phase1_eval_phi4_qlora.json

📊  PHASE 1 SUMMARY — PHI4_QLORA
  Scored      : 150 / 150 samples
  Overall avg : 2.48 / 5.0

  Score distribution:
    5/5  █████████████████████████████████ (33)
    4/5  ███████████ (11)
    3/5  ██ (2)
    2/5  █████████████████████████████████████████████████████ (53)
    1/5  ███████████████████████████████████████████████████ (51)

  By task type:
    Act Identification   avg=4.46  (n=39)
    Direct Q&A           avg=1.23  (n=31)
    Explanation          avg=2.31  (n=59)
    Summarization        avg=1.14  (n=21)
